In [1]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END

In [2]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]] # Union means each item of list will be of either HumanMessage type or AIMessage type

In [3]:
llm = ChatOllama(model = "qwen2.5")

In [18]:
def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    response = llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content = response.content))

    print(f"\nAI: {response.content}")
    print(f"\nCurrent State: ", state["messages"])
    return state

In [19]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

In [23]:
conversation_history = []

user_input = input("Enter: ")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    converation_history = result["messages"]
    user_input = input("Enter: ")

Enter:  Hi, nice to meet you!



AI: Hello! Nice to meet you too. How can I assist you today?

Current State:  [HumanMessage(content='Hi, nice to meet you!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Nice to meet you too. How can I assist you today?', additional_kwargs={}, response_metadata={})]


Enter:  My name is Ali



AI: Hello Ali! It's nice to meet you. How can I assist you today? Do you have any specific questions or topics you'd like to discuss?

Current State:  [HumanMessage(content='Hi, nice to meet you!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Nice to meet you too. How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Ali', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Ali! It's nice to meet you. How can I assist you today? Do you have any specific questions or topics you'd like to discuss?", additional_kwargs={}, response_metadata={})]


Enter:  who am I?



AI: You're Ali, according to your introduction. Is there anything specific you'd like to know or discuss further?

Current State:  [HumanMessage(content='Hi, nice to meet you!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! Nice to meet you too. How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My name is Ali', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Ali! It's nice to meet you. How can I assist you today? Do you have any specific questions or topics you'd like to discuss?", additional_kwargs={}, response_metadata={}), HumanMessage(content='who am I?', additional_kwargs={}, response_metadata={}), AIMessage(content="You're Ali, according to your introduction. Is there anything specific you'd like to know or discuss further?", additional_kwargs={}, response_metadata={})]


Enter:  exit


# Store Memory in a text file

In [24]:
# File
with open("logging.txt", "w") as file:
    file.write("Your Conversation Log: \n")
    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
    file.write("End of Conversation")
print("Conversation saved to logging.txt")

Conversation saved to logging.txt


# Store Memory in an Efficient way
- You are storing a lot of previous unnecessory conversations;
- You may make a limit like if user_input > 5 then you remove first conversation;